In [10]:
import torch
import torch.nn as nn
from ConditionalImitationLearningNetwork import ImitationLearningNetwork_Training
from CarlaDatasetLoader import get_custom_dataloader, get_custom_dataset, CarlaDataset
from utils import save_model, load_model, jit_compile_model
from torchvision import transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import tqdm
from torch.utils.data import random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
model = ImitationLearningNetwork_Training()
model = jit_compile_model(model).to(device)

In [3]:
#len(dataset) = 657_600 --> batch_size = 200 ---> train/test split = ~.1?
EPOCHS = 100
criterion = nn.MSELoss()
batch_size = 64
train_dataloader = get_custom_dataloader()
valid_dataloader = get_custom_dataloader(CarlaDataset("data/data.csv", for_training=False))
optimizer = torch.optim.Adam(model.parameters())


In [41]:
657600 / 200

3288.0